# Predicting Irish house energy ratings using supervised machine learning methods 

## AIB Datahack 2016

The exercise in part 1 of this series, where can be viewed [here](https://nbviewer.jupyter.org/github/mcsweena/AIB-Datahack-2016/blob/master/Part1-Initial-Data-Exploration.ipynb), was very useful. The analysis highlighted some very important insights into the variables I believed to be most important. 

But what about the variables I didn't feel were as important, what if secretly, they're the lynch pin to this whole thing. Lets use a few different techniques to see if we can uncover any of these hidden important variables.

First, to bring as much data into focus as possible, lets do some light processing

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
# Data Import
train = pd.read_csv("large_data/training.txt", sep='\t', encoding='ISO-8859-1', header=0, low_memory=False, index_col=0)
test = pd.read_csv("large_data/testing.txt", sep='\t', encoding='ISO-8859-1', header=0, low_memory=False, index_col=0)

train.head(3)

In [ ]:
# Dataframe shapes
print("Train: ", train.shape)
print("Test: ", test.shape)

In [ ]:
# Lets check the columns
train.columns